# Part 1: Data Model Generation

[![Notebook](https://shields.io/badge/notebook-access-green?logo=jupyter&style=for-the-badge)](https://github.com/cognitedata/neat/blob/main/docs/tutorial/notebooks/part-1-data-model-generation.ipynb)

* author: Nikola Vasiljevic
* date: 2023-05-31

This notebook represent Part 1 of NEAT Onboarding tutorial. In this notebook we will demonstrate how to generate data model using NEAT.
All you need is basic knowledge of Excel and Python, and very good knowledge of domain you are trying to model.

For simplicity we will define a simple power grid data model.

Before proceeding download `Transformation Rule` template using [this link](https://drive.google.com/uc?export=download&id=1yJxK35IaKVpZJas60ojReCjh-Ppj9fKX). Unzip file and open template:


<video src="../../videos/tutorial-1-download-rules-template.mp4" controls>
</video>


> Gif playing too fast, [check video version of the above animated gif](https://drive.google.com/file/d/1_ipfeFauaGvJC4JTL2klQyt7AK68SWnn/preview)!

Let's import all necessary libraries:

In [ ]:
from pathlib import Path
from cognite.neat.core import loader, parser, extractors
from cognite.neat.core.utils import remove_namespace, add_triples
from cognite.neat.core.mocks.graph import generate_triples

%reload_ext autoreload
%autoreload 2

Let's now fill in the template sheet, going sheet by sheet in the following order
- `Metadata` : where we will provide metadata about data model itself
- `Classes` : where we will defined classes
- `Properties`: where we will define properties for each of defined classes


<video src="../../videos/tutorial-1-defining-data-model.mp4" controls>
</video>


For more information about `Transformation Rules` check [this detail overview](../../transformation-rules.md). 


Once we are done with filling in the template sheet, let's load it and transform it to GraphQL schema which represents our data model:

In [85]:
TRANSFORMATION_RULES = Path("insert_path_to_rules_sheet")
raw_sheets = loader.rules.excel_file_to_table_by_name(TRANSFORMATION_RULES)
transformation_rules = parser.parse_transformation_rules(raw_sheets)
data_model_gql = extractors.rules2graphql_schema(transformation_rules)

If we now print derive GraphQL schema we can see how each of the objects (i.e. classes) are defined and represented in GraphQL:

In [87]:
print(data_model_gql)

type GeographicalRegion {
  name: String!
}

type SubGeographicalRegion {
  name: String!
  geographicalRegion: GeographicalRegion
}

type Substation {
  name: String!
  subGeographicalRegion: SubGeographicalRegion
}

type Terminal {
  name: String!
  aliasName: [String]
  substation: Substation
}


Derive GraphQL schema now can be uploaded to CDF and resolved as Flexible Data Model:

<video src="../../videos/tutorial-1-upload-gql-schema-to-cdf.mp4" controls>
</video>
